In [1]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import datetime

import requests
import json

In [2]:
orders = []
page = 1
while True:
    url = f"https://bling.com.br/Api/v2/pedidos/page={page}/json/"
    params = {
        "apikey": "9e4f7df3bcb41a13ed8746d50556e66cf16af5fe3a48e62f4704e7d86230acd724814239"
    }
    response = requests.get(url, params=params)
    json_data = json.loads(response.text)
    if (
        "retorno" in json_data
        and "pedidos" in json_data["retorno"]
        and json_data["retorno"]["pedidos"]
    ):
        for order in json_data["retorno"]["pedidos"]:
            order_data = {
                "numero": order["pedido"]["numero"],
                "data": order["pedido"]["data"],
                "cliente": order["pedido"]["cliente"]["nome"],
                "valor": order["pedido"]["totalvenda"],
                "situacao": order["pedido"]["situacao"],
            }
            if "tipoIntegracao" in order["pedido"]:
                order_data["tipoIntegracao"] = order["pedido"]["tipoIntegracao"]
            else:
                order_data["tipoIntegracao"] = None
            orders.append(order_data)
        page += 1
    else:
        break

In [18]:
df = pd.DataFrame(orders)

df['valor'] = df['valor'].astype(float)

df_cru = df.copy()
df_cru.loc[df_cru["cliente"] == "Consumidor Final", "cliente"] = "Lojinha"
df_cru.loc[df_cru["tipoIntegracao"] == "WooCommerceWH", "cliente"] = "Site"
df_cru.loc[df_cru["cliente"] == "Venda Externa", "cliente"] = "Venda Mauricio"
df_cru.loc[df_cru["situacao"] != "Cancelado", "situacao"] = 1

df_cru["data"] = pd.to_datetime(df_cru["data"])
df_cru["dia"] = df_cru["data"].dt.day
df_cru["mes"] = df_cru["data"].dt.month
df_cru["ano"] = df_cru["data"].dt.year


df_graph1 = df_cru.groupby(["cliente", "dia", "mes"])["valor"].sum().reset_index()

fig_filtrado = go.Figure()

for cliente in df_graph1["cliente"].unique():
    data = df_graph1[df_graph1["cliente"] == cliente]
    fig_filtrado.add_trace(
        go.Scatter(
            x=data["dia"],
            y=data["valor"],
            mode="lines+markers+text",  # Adicionar o modo de exibição de texto
            # Adicionar o valor como texto em cada ponto
            text=[f"R${round(valor, 2):.2f}" for valor in data["valor"]],
            textposition="top center",  # Definir a posição do texto
            name=cliente,
        )
    )
fig_filtrado